In [1]:
using ForwardDiff, Colors
using ForwardDiff: Partials, Dual, value, partials

In [2]:
struct Tag end

In [3]:
x = Dual{Tag}(1.0, (1.0, ))

Dual{Tag}(1.0,1.0)

In [4]:
f = x -> x^2 + 2x

#11 (generic function with 1 method)

In [5]:
f(x)

Dual{Tag}(3.0,4.0)

In [6]:
partials(f(x))

1-element Partials{1, Float64}:
 4.0

In [7]:
function g(x)
    RGB(x,2*x,x^2)
end

g (generic function with 1 method)

In [8]:
@show g(1.0)
nothing

g(1.0) = RGB{Float64}(1.0, 2.0, 1.0)


In [9]:
@show g(x)
nothing

ArgumentError: ArgumentError: component type FixedPointNumbers.N0f8 is an 8-bit type representing 256 values from 0.0 to 1.0,
  but the values (Dual{Tag}(1.0,1.0), Dual{Tag}(2.0,2.0), Dual{Tag}(1.0,2.0)) do not lie within this range.
  See the READMEs for FixedPointNumbers and ColorTypes for more information.

In [18]:
typeof(x)

Dual{Tag, Float64, 1}

In [24]:
using ForwardDiff, Colors
using ForwardDiff: Dual
struct Tag end
x = Dual{Tag}(1.0, (1.0, ))
RGB{typeof(x)}(x, 2*x, x^2)


TypeError: TypeError: in RGB, in T, expected T<:Union{AbstractFloat, FixedPointNumbers.FixedPoint}, got Type{Dual{Tag, Float64, 1}}

In [10]:
using StaticArrays

In [11]:
function h(x)
    SA[x,2*x,x^2]
end

h (generic function with 1 method)

In [12]:
h(1.0)

3-element SVector{3, Float64} with indices SOneTo(3):
 1.0
 2.0
 1.0

In [13]:
h(x)

3-element SVector{3, Dual{Tag, Float64, 1}} with indices SOneTo(3):
 Dual{Tag}(1.0,1.0)
 Dual{Tag}(2.0,2.0)
 Dual{Tag}(1.0,2.0)

In [14]:
using CUDA

In [15]:
function mapper(f, x, y)
    @assert length(x) == length(y)
    r = Int32.(length(x))
    i = threadIdx().x + (blockIdx().x - 1) * blockDim().x
    
    @inbounds if i <= r 
        @inline y[i] = f(x[i])
    end
    return
end


mapper (generic function with 1 method)

In [16]:
x = CuArray(1:10) * 1.0
y = similar(x)

10-element CuArray{Float64, 1, CUDA.DeviceMemory}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [17]:
@show threads = 256
@show blocks = cld.(size(x), threads)

threads = 256 = 256
blocks = cld.(size(x), threads) = (1,)


(1,)

In [18]:
@cuda threads=threads blocks=blocks mapper( x->x+2, x, y)
CUDA.synchronize()

In [19]:
y

10-element CuArray{Float64, 1, CUDA.DeviceMemory}:
  3.0
  4.0
  5.0
  6.0
  7.0
  8.0
  9.0
 10.0
 11.0
 12.0

In [20]:
_x = Dual{Tag}(1.0, (1.0, ))

Dual{Tag}(1.0,1.0)

In [31]:
xx = x .+ _x

10-element CuArray{Dual{Tag, Float64, 1}, 1, CUDA.DeviceMemory}:
  Dual{Tag}(2.0,1.0)
  Dual{Tag}(3.0,1.0)
  Dual{Tag}(4.0,1.0)
  Dual{Tag}(5.0,1.0)
  Dual{Tag}(6.0,1.0)
  Dual{Tag}(7.0,1.0)
  Dual{Tag}(8.0,1.0)
  Dual{Tag}(9.0,1.0)
 Dual{Tag}(10.0,1.0)
 Dual{Tag}(11.0,1.0)

In [32]:
yy = similar(xx)

10-element CuArray{Dual{Tag, Float64, 1}, 1, CUDA.DeviceMemory}:
 Dual{Tag}(0.0,0.0)
 Dual{Tag}(0.0,0.0)
 Dual{Tag}(0.0,0.0)
 Dual{Tag}(0.0,0.0)
 Dual{Tag}(0.0,0.0)
 Dual{Tag}(0.0,0.0)
 Dual{Tag}(0.0,0.0)
 Dual{Tag}(0.0,0.0)
 Dual{Tag}(0.0,0.0)
 Dual{Tag}(0.0,0.0)

In [35]:
@cuda threads=threads blocks=blocks mapper( x->(x+2)^2 - exp(x), xx, yy)
CUDA.synchronize()

In [36]:
yy

10-element CuArray{Dual{Tag, Float64, 1}, 1, CUDA.DeviceMemory}:
      Dual{Tag}(8.61094390106935,0.6109439010693496)
      Dual{Tag}(4.914463076812332,-10.085536923187668)
    Dual{Tag}(-18.598150033144236,-42.598150033144236)
    Dual{Tag}(-99.4131591025766,-134.4131591025766)
   Dual{Tag}(-339.4287934927351,-387.4287934927351)
  Dual{Tag}(-1015.6331584284585,-1078.6331584284585)
  Dual{Tag}(-2880.9579870417283,-2960.9579870417283)
  Dual{Tag}(-7982.083927575384,-8081.083927575384)
 Dual{Tag}(-21882.465794806714,-22002.465794806714)
 Dual{Tag}(-59705.14171519782,-59848.14171519782)

In [38]:
@cuda threads=threads blocks=blocks mapper( x->((x + 0.001) % 2.0)^2 , xx, yy)
CUDA.synchronize()
yy

10-element CuArray{Dual{Tag, Float64, 1}, 1, CUDA.DeviceMemory}:
 Dual{Tag}(9.999999999997797e-7,0.0019999999999997797)
 Dual{Tag}(1.0020009999999997,2.002)
 Dual{Tag}(1.0000000000006678e-6,0.002000000000000668)
 Dual{Tag}(1.0020010000000006,2.0020000000000007)
 Dual{Tag}(1.0000000000006678e-6,0.002000000000000668)
 Dual{Tag}(1.0020010000000006,2.0020000000000007)
 Dual{Tag}(9.999999999988916e-7,0.0019999999999988916)
 Dual{Tag}(1.0020009999999988,2.001999999999999)
 Dual{Tag}(9.999999999988916e-7,0.0019999999999988916)
 Dual{Tag}(1.0020009999999988,2.001999999999999)